In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, length, col
from pyspark.sql.functions import avg
from pyspark.sql.functions import count, desc
from pyspark.sql.functions import sum as _sum, when
from pyspark.sql.functions import split, col, explode, length, regexp_replace

In [41]:
def LongWord(path_file):
  spark = SparkSession.builder.appName("LongestWord").getOrCreate()

  data = spark.read.text(path_file) \
    .select(split(col("value"), "\t").alias("cols")) \
    .select(col("cols")[2].alias("text"))

  words = data.select(explode(split(regexp_replace(col("text"), "[^a-zA-Zа-яА-Я-]", " "), " ")).alias("word")) \
      .filter(col("word") != "")

  max_len = words.select(length(col("word")).alias("len")).agg({"len": "max"}).collect()[0][0]
  longest_word = words.filter(length(col("word")) == max_len).first()["word"]

  print(f"Самое длинное слово: '{longest_word}' (длина: {max_len} символов)")

  return words

In [42]:
def average_len_word(words):
  avg_len = words.select(length(col("word")).alias("len")).agg({"len": "avg"}).collect()[0][0]
  print(f"Средняя длина слова: {avg_len:.2f} символов")

In [43]:
def terms_word(words):
  latin_word = words.filter(col("word").rlike("^[a-zA-Z]+$")) \
      .groupBy("word").agg(count("*").alias("cnt")) \
      .orderBy(desc("cnt")).first()

  print(f"Самое частотное слово (латиница): '{latin_word['word']}' (встречается {latin_word['cnt']} раз)")

In [44]:
def title_word(words):
  word_stats = words.groupBy("word").agg(
      count("*").alias("total_count"),
      _sum(when(col("word").rlike("^[А-ЯA-Z]"), 1).otherwise(0)).alias("upper_count")
  )

  result = word_stats.filter(
      (col("total_count") > 10) &
      (col("upper_count") / col("total_count") > 0.5)
  ).collect()

  print("Слова, которые чаще пишутся с заглавной буквы:")
  for row in result:
      print(f"'{row['word']}': {row['upper_count']}/{row['total_count']} раз")

In [45]:
def reduction_word_1(words):
  abbr1 = words.filter(col("word").rlike("^[а-яА-Я]{1,3}\\.$")) \
    .groupBy("word").agg(count("*").alias("cnt")) \
    .orderBy(desc("cnt")).limit(5).collect()

  print("Устойчивые сокращения (пр., др. и т.д.):")
  for row in abbr1:
      print(f"'{row['word']}': {row['cnt']} раз")

In [48]:
def reduction_word_2(words):
  abbr2 = words.filter(col("word").rlike("^[а-яА-Я]\\.[а-яА-Я]\\.$")) \
    .groupBy("word").agg(count("*").alias("cnt")) \
    .orderBy(desc("cnt")).limit(5).collect()

  print("Устойчивые сокращения (т.п., н.э. и т.д.):")
  print(abbr2)
  for row in abbr2:
      print(f"'{row['word']}': {row['cnt']} раз")

In [47]:
words = LongWord("/content/wiki.txt")
print("-"*20)
average_len_word(words)
print("-"*20)
terms_word(words)
print("-"*20)
title_word(words)
print("-"*20)
reduction_word_1(words)
print("-"*20)
reduction_word_2(words)
print("-"*20)

Выходные данные были обрезаны до нескольких последних строк (5000).
'Романом': 18/18 раз
'Эсхил': 13/13 раз
'Чили': 147/147 раз
'G': 342/342 раз
'Вятская': 18/18 раз
'Парламенте': 17/17 раз
'Кристи': 26/26 раз
'Константинопольский': 24/24 раз
'Город-Герой': 11/11 раз
'КХЛ': 14/14 раз
'ИЮПАК': 38/38 раз
'Или': 37/37 раз
'Черняховского': 17/17 раз
'Социализм': 16/16 раз
'Нерона': 80/80 раз
'Христова': 71/71 раз
'Калифорнии': 131/131 раз
'ЯМР': 12/12 раз
'Краткий': 19/19 раз
'Дали': 12/12 раз
'Лоренца': 169/169 раз
'Дзержинского': 43/43 раз
'Львиное': 14/14 раз
'Плоды': 14/14 раз
'Линда': 24/24 раз
'Афина': 62/62 раз
'Про': 32/32 раз
'Бернулли': 66/66 раз
'Software': 92/92 раз
'Сервис': 11/11 раз
'MS-DOS': 96/96 раз
'MediaWiki': 23/23 раз
'Сотрудничал': 11/11 раз
'УПИ': 12/12 раз
'Петропавловск': 15/15 раз
'Назаров': 15/15 раз
'Тайга': 22/22 раз
'Эйр': 17/17 раз
'Марком': 35/35 раз
'Дополнительным': 13/13 раз
'Нейман': 13/13 раз
'Практическая': 11/11 раз
'Танзанией': 12/12 раз
'Пьером': 2

In [49]:
reduction_word_2(words)

Устойчивые сокращения (т.п., н.э. и т.д.):
[]
